In [280]:
import pandas as pd
import json
import texthero as hero
from texthero import preprocessing
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import seaborn as sns
import nltk, string, numpy as np

# Reference links

https://towardsdatascience.com/natural-language-processing-count-vectorization-with-scikit-learn-e7804269bb5e

https://sites.temple.edu/tudsc/2017/03/30/measuring-similarity-between-texts-in-python/ (cosin similarity)

https://towardsdatascience.com/machine-learning-text-processing-1d5a2d638958

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4119097/

https://dataconomy.com/2015/04/implementing-the-five-most-popular-similarity-measures-in-python/  (jaccard similarity)

https://machinelearningmastery.com/clean-text-machine-learning-python/#:~:text=Python%20offers%20a%20function%20called,set%20of%20characters%20to%20another.&text=We%20can%20put%20all%20of,word%20to%20remove%20the%20punctuation. (remove puntuations)

https://nlp.stanford.edu/IR-book/html/htmledition/hierarchical-agglomerative-clustering-1.html#:~:text=Bottom%2Dup%20hierarchical%20clustering%20is,until%20individual%20documents%20are%20reached.(aggolmatarive algoritm steps)

http://rstudio-pubs-static.s3.amazonaws.com/265632_3ad9e0b981244e15887677f8dffb39a0.html (methods for optimal clusters)

https://help.relativity.com/9.4/Content/Relativity/Analytics/Cluster_visualization.htm#Circle2 (cluster viz)

https://texthero.org/docs/api/texthero.representation.term_frequency (text hero)

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4119097/(clustering paper)


In [281]:
patient=pd.read_csv('Trial Match Master File.csv')

In [282]:
patient.head()

,NCI_ID,NCT_ID,Patient_ID
0,NCI-2009-00336,NCT00392327,1
1,NCI-2011-00878,NCT00956007,1
2,NCI-2011-02500,NCT01366144,1
3,NCI-2011-02674,NCT01368588,1
4,NCI-2012-01695,NCT01775475,1


In [283]:
patient_new=patient.rename({'NCT_ID':'NCTId'}, axis=1)

In [284]:
patient_new.head()

,NCI_ID,NCTId,Patient_ID
0,NCI-2009-00336,NCT00392327,1
1,NCI-2011-00878,NCT00956007,1
2,NCI-2011-02500,NCT01366144,1
3,NCI-2011-02674,NCT01368588,1
4,NCI-2012-01695,NCT01775475,1


In [285]:
df_patient = pd.merge(patient_new, description, on='NCTId')

In [286]:
df_patient.head(10)

,NCI_ID,NCTId,Patient_ID,index,Inclusion,Exclusion,description,clean_text,tfidf,tfidf_threshold,cluster,pca


In [287]:
given_data=pd.read_csv('Dataset1_Hemoglobin_Trials_First.csv')

In [288]:
given_data.head()

,nci_id,nct_id,official_title,inclusion_indicaInclusionor,description,Text,NCIT
0,NCI-2009-00336,NCT00392327,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
1,NCI-2009-00470,NCT00632853,Phase III Comparison of Thoracic Radiotherapy ...,NaN,NaN,NaN,NaN
2,NCI-2009-00595,NCT00719303,Can Diet and Physical Activity Modulate Ovaria...,NaN,NaN,NaN,NaN
3,NCI-2009-00603,NCT00492778,A Randomized Trial of Pelvic Irradiation with ...,NaN,NaN,NaN,NaN
4,NCI-2009-00817,NCT00001337,Dose-Adjusted EPOCH Chemotherapy and Rituximab...,NaN,NaN,NaN,NaN


In [289]:
given_data_new=given_data.rename({'nct_id':'NCTId'}, axis=1)
given_data_new.head()

,nci_id,NCTId,official_title,inclusion_indicaInclusionor,description,Text,NCIT
0,NCI-2009-00336,NCT00392327,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
1,NCI-2009-00470,NCT00632853,Phase III Comparison of Thoracic Radiotherapy ...,NaN,NaN,NaN,NaN
2,NCI-2009-00595,NCT00719303,Can Diet and Physical Activity Modulate Ovaria...,NaN,NaN,NaN,NaN
3,NCI-2009-00603,NCT00492778,A Randomized Trial of Pelvic Irradiation with ...,NaN,NaN,NaN,NaN
4,NCI-2009-00817,NCT00001337,Dose-Adjusted EPOCH Chemotherapy and Rituximab...,NaN,NaN,NaN,NaN


In [324]:
given_data_new=given_data_new.dropna(subset=['description'])
given_data_new.head()

,nci_id,NCTId,official_title,inclusion_indicaInclusionor,description,Text,NCIT
0,NCI-2009-00336,NCT00392327,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
6,NCI-2011-00312,NCT01012817,"A Phase 1/2 Trial of ABT-888, an Inhibitor of ...",Inclusion,Hemoglobin >= 9.0 g/dL,Hemoglobin measurement greater than or equal t...,(C64848 >= 9g/dL)
7,NCI-2011-00878,NCT00956007,A Phase III Study of Postoperative Radiation T...,Inclusion,Hemoglobin >= 8.0 g/dL (note: the use of trans...,Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
8,NCI-2011-01915,NCT00887146,N0577 (CODEL): Phase III Intergroup Study of R...,Inclusion,Hemoglobin (Hgb) > 9.0 g/dL obtained =< 21 day...,Hemoglobin measurement greater than 9 grams p...,(C64848 > 9g/dL)
10,NCI-2011-01973,NCT00980954,Phase III Randomized Study of Concurrent Chemo...,Inclusion,Hemoglobin >= 10.0 g/dl (Note: the use of tran...,Hemoglobin measurement greater than or equal t...,(C64848 >= 10g/dL)


In [294]:
df_patient_new = pd.merge(patient_new, given_data_new, on='NCTId')

In [295]:
df_patient_new

,NCI_ID,NCTId,Patient_ID,nci_id,official_title,inclusion_indicaInclusionor,description,Text,NCIT
0,NCI-2009-00336,NCT00392327,1,NCI-2009-00336,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
1,NCI-2009-00336,NCT00392327,2,NCI-2009-00336,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
2,NCI-2009-00336,NCT00392327,3,NCI-2009-00336,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
3,NCI-2009-00336,NCT00392327,4,NCI-2009-00336,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
4,NCI-2009-00336,NCT00392327,5,NCI-2009-00336,Efficacy of Carboplatin Administered Concomita...,Inclusion,Hemoglobin >= 8 g/dl (may be transfused),Hemoglobin measurement greater than or equal t...,(C64848 >= 8g/dL)
...,...,...,...,...,...,...,...,...,...
22081,NCI-2015-01408,NCT02535325,96,NCI-2015-01408,A Phase I Study of Methoxyamine Combined with ...,Inclusion,Hemoglobin >= 9 g/dL without transfusion withi...,Hemoglobin measurement greater than or equal t...,((C64848 >= 9g/dL) AND (C15192 = NO OR C15192...
22082,NCI-2015-01408,NCT02535325,97,NCI-2015-01408,A Phase I Study of Methoxyamine Combined with ...,Inclusion,Hemoglobin >= 9 g/dL without transfusion withi...,Hemoglobin measurement greater than or equal t...,((C64848 >= 9g/dL) AND (C15192 = NO OR C15192...
22083,NCI-2015-01408,NCT02535325,98,NCI-2015-01408,A Phase I Study of Methoxyamine Combined with ...,Inclusion,Hemoglobin >= 9 g/dL without transfusion withi...,Hemoglobin measurement greater than or equal t...,((C64848 >= 9g/dL) AND (C15192 = NO OR C15192...
22084,NCI-2015-01408,NCT02535325,99,NCI-2015-01408,A Phase I Study of Methoxyamine Combined with ...,Inclusion,Hemoglobin >= 9 g/dL without transfusion withi...,Hemoglobin measurement greater than or equal t...,((C64848 >= 9g/dL) AND (C15192 = NO OR C15192...


In [296]:
pat_dec=df_patient_new[['NCTId', 'Text','Patient_ID']].copy()
pat_dec.head()

,NCTId,Text,Patient_ID
0,NCT00392327,Hemoglobin measurement greater than or equal t...,1
1,NCT00392327,Hemoglobin measurement greater than or equal t...,2
2,NCT00392327,Hemoglobin measurement greater than or equal t...,3
3,NCT00392327,Hemoglobin measurement greater than or equal t...,4
4,NCT00392327,Hemoglobin measurement greater than or equal t...,5


In [297]:
pat_dec[(pat_dec['Patient_ID'] == 1)]

,NCTId,Text,Patient_ID
0,NCT00392327,Hemoglobin measurement greater than or equal t...,1
95,NCT00956007,Hemoglobin measurement greater than or equal t...,1
190,NCT01366144,Hemoglobin measurement greater than or equal t...,1
375,NCT01368588,Hemoglobin measurement greater than or equal t...,1
560,NCT01775475,Hemoglobin measurement greater than or equal t...,1
...,...,...,...
20760,NCT02595866,Hemoglobin measurement greater than 9 grams pe...,1
20996,NCT01573442,Hemoglobin measurement greater than 11 grams ...,1
21456,NCT01587352,Hemoglobin measurement greater than or equal t...,1
21726,NCT02101775,Hemoglobin measurement greater than or equal t...,1


In [298]:
given_data1=pd.read_csv('Dataset1_Hemoglobin_Trials_First.csv')
given_data2=pd.read_csv('Dataset1_HIV_Trials_First.csv')
given_data3=pd.read_csv('Dataset1_Performance_Status_Trials_First.csv')
given_data4=pd.read_csv('Dataset1_Platelets_Trials_First.csv')
given_data5=pd.read_csv('Dataset1_Prior_Therapy_Trials_First.csv')
given_data6=pd.read_csv('Dataset1_WBC_Trials_First.csv')

In [299]:
given_data_new1=given_data1.rename({'nct_id':'NCTId'}, axis=1)
given_data_new2=given_data2.rename({'nct_id':'NCTId'}, axis=1)
given_data_new3=given_data3.rename({'nct_id':'NCTId'}, axis=1)
given_data_new4=given_data3.rename({'nct_id':'NCTId'}, axis=1)
given_data_new5=given_data4.rename({'nct_id':'NCTId'}, axis=1)
given_data_new6=given_data6.rename({'nct_id':'NCTId'}, axis=1)

In [300]:
df_patient_new1 = pd.merge(patient_new, given_data_new1, on='NCTId')
df_patient_new2 = pd.merge(patient_new, given_data_new2, on='NCTId')
df_patient_new3= pd.merge(patient_new, given_data_new3, on='NCTId')
df_patient_new4 = pd.merge(patient_new, given_data_new4, on='NCTId')
df_patient_new5 = pd.merge(patient_new, given_data_new5, on='NCTId')
df_patient_new6 = pd.merge(patient_new, given_data_new6, on='NCTId')

In [301]:
given_data_new1=given_data_new1.dropna(subset=['description'])
given_data_new2=given_data_new2.dropna(subset=['description'])
given_data_new3=given_data_new3.dropna(subset=['description'])
given_data_new4=given_data_new4.dropna(subset=['description'])
given_data_new5=given_data_new6.dropna(subset=['description'])
given_data_new6=given_data_new.dropna(subset=['description'])


In [305]:
pat_dec1=df_patient_new1[['NCTId', 'Text','Patient_ID']].copy()
pat_dec2=df_patient_new2[['NCTId', 'Text','Patient_ID']].copy()
pat_dec3=df_patient_new3[['NCTId', 'Text','Patient_ID']].copy()
pat_dec4=df_patient_new4[['NCTId', 'Text','Patient_ID']].copy()
pat_dec5=df_patient_new5[['NCTId', 'Text','Patient_ID']].copy()
pat_dec6=df_patient_new6[['NCTId', 'Text','Patient_ID']].copy()

In [123]:
df_patient_new6.columns

Index(['NCI_ID', 'NCTId', 'Patient_ID', 'nci_id', 'official_title',
       'inclusion_indicaInclusionor', 'description', 'Text', 'NCIT'],
      dtype='object')

In [303]:
df_patient_new3=df_patient_new3.rename({'text':'Text'}, axis=1)

In [304]:
df_patient_new4=df_patient_new4.rename({'text':'Text'}, axis=1)

In [240]:
all_patients=pd.concat([pat_dec,pat_dec1,pat_dec2,pat_dec3,pat_dec4,pat_dec5,pat_dec6]).reset_index()

# Taking patient 1 details to another data frame

In [306]:
patient_1=all_patients[(all_patients['Patient_ID'] == 1)];

In [307]:
patient_1=patient_1.drop(['Patient_ID', 'index'],axis=1)

In [ ]:
patient_1['Text'] = patient_1['Text'].astype('str') 

In [310]:
grp_patient=patient_1.groupby(['NCTId'])['Text'].agg(lambda x: ','.join(x)) 

In [243]:
patient_1.dtypes

NCTId    object
Text     object
dtype: object

In [311]:
patient_1.head()

,NCTId,Text
0,NCT00392327,Hemoglobin measurement greater than or equal t...
95,NCT00956007,Hemoglobin measurement greater than or equal t...
190,NCT01366144,Hemoglobin measurement greater than or equal t...
375,NCT01368588,Hemoglobin measurement greater than or equal t...
560,NCT01775475,Hemoglobin measurement greater than or equal t...


In [312]:
grp_patient=grp_patient.reset_index()

In [313]:
grp_patient.head()

,NCTId,Text
0,NCT00392327,Hemoglobin measurement greater than or equal t...
1,NCT00956007,Hemoglobin measurement greater than or equal t...
2,NCT01089101,Hemoglobin measurement greater than or equal t...
3,NCT01220583,Hemoglobin measurement greater than or equal t...
4,NCT01273155,"nan,(HIV=NO) OR (HIV =UNKNOWN) OR (HIV=YES AND..."


In [314]:
custom_pipeline1 = [preprocessing.fillna,
                   preprocessing.lowercase,
                   preprocessing.remove_whitespace,
                  preprocessing.remove_punctuation,
                  preprocessing.remove_angle_brackets,
                  preprocessing.remove_brackets,
                  preprocessing.remove_round_brackets,
                  preprocessing.remove_html_tags,
                  preprocessing.remove_stopwords
                  ]
grp_patient['clean'] = grp_patient['Text'].pipe(hero.clean,custom_pipeline1)

In [265]:
#Normalize by lemmatization:

nltk.download('wordnet') # first-time use only
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sree\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [315]:
#Normalize by lemmatization:

nltk.download('wordnet') # first-time use only
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sree\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [316]:
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
def jaccard_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    tfidf =tfidf.toarray()
    return pairwise_distances(tfidf,metric='jaccard')
A=jaccard_similarity(grp_patient['clean'])

C:\Users\Sree\Anaconda3\envs\clinical_challenge\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.

C:\Users\Sree\Anaconda3\envs\clinical_challenge\lib\site-packages\sklearn\metrics\pairwise.py:1735: DataConversionWarning:

Data was converted to boolean for metric jaccard



In [317]:
grp_patient['tfidf'] = pd.Series(list(A))

In [318]:
hc = AgglomerativeClustering(n_clusters = 15, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
b_hc=hc.fit_predict(A)

C:\Users\Sree\Anaconda3\envs\clinical_challenge\lib\site-packages\scipy\cluster\hierarchy.py:830: ClusterWarning:

scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix



In [319]:
grp_patient['cluster1']=b_hc.astype(str)

In [320]:
grp_patient.groupby(['cluster1']).count()

,NCTId,Text,clean,tfidf
cluster1,,,,
0,10,10,10,10
1,7,7,7,7
10,3,3,3,3
11,1,1,1,1
12,7,7,7,7
13,2,2,2,2
14,4,4,4,4
2,2,2,2,2
3,9,9,9,9


In [321]:
grp_patient.to_csv('patient_trial_1.csv', index=False)